# Objective

Establish the accuracy of a base LLM (Mistral) on dialogue summarization


# Setup

In [1]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps -q "xformers==0.0.25" trl peft accelerate bitsandbytes # Downgrade xformers to 0.0.25

!pip install -q datasets evaluate bert-score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.8 MB/s e

# Imports

In [2]:
import torch
import evaluate

from datasets import load_dataset
from tqdm import tqdm
import importlib

# Import unsloth after installing necessary packages.
import unsloth
from unsloth import FastLanguageModel # Import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.2.1+cu121 with CUDA 1201 (you have 2.5.1+cu121)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


# Test Data

Source: https://huggingface.co/datasets/knkarthick/dialogsum

In [3]:
dataset = load_dataset("knkarthick/dialogsum")

README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [4]:
test_data_size = 30

In [5]:
test_dataset = dataset['test'].shuffle(seed=42).select(range(test_data_size))

In [6]:
test_dialogues = [sample['dialogue'] for sample in test_dataset]
test_summaries = [sample['summary'] for sample in test_dataset]

In [7]:
print(test_dataset[0])

{'id': 'test_263_2', 'dialogue': "#Person1#: Hi! What are you watching?\n#Person2#: It's a program about islam. It's very interesting.\n#Person1#: Wow! So many people! Where are they and what are they doing?\n#Person2#: They are muslims on a pilgrimage to mecca. Muslims call this pilgrimage 'haj'.\n#Person1#: Why do they go there?\n#Person2#: Muslims believe that every man who is able should go on a haj at least once in his life. Mecca is the spiritual centre of the muslim faith.\n#Person1#: When muslims pray, they face towards mecca.\n#Person2#: That's right. Unfortunately, so many people go on the haj each year that there are often stamped and people get killed.\n#Person1#: I heard about that. The pilgrims must walk around a large, sacred black stone.\n#Person2#: That's right. That's when accidents often happen. The Saudi government tries to limit the number of pilgrims, to reduce the chances of accidents.\n#Person1#: Pilgrimages are common in many faiths.\n#Person2#: Yes. In England

In [8]:
print(f"Number of test dialogues: {len(test_dialogues)}")
print(test_dialogues[:5])  # Print a sample

Number of test dialogues: 30
["#Person1#: Hi! What are you watching?\n#Person2#: It's a program about islam. It's very interesting.\n#Person1#: Wow! So many people! Where are they and what are they doing?\n#Person2#: They are muslims on a pilgrimage to mecca. Muslims call this pilgrimage 'haj'.\n#Person1#: Why do they go there?\n#Person2#: Muslims believe that every man who is able should go on a haj at least once in his life. Mecca is the spiritual centre of the muslim faith.\n#Person1#: When muslims pray, they face towards mecca.\n#Person2#: That's right. Unfortunately, so many people go on the haj each year that there are often stamped and people get killed.\n#Person1#: I heard about that. The pilgrims must walk around a large, sacred black stone.\n#Person2#: That's right. That's when accidents often happen. The Saudi government tries to limit the number of pilgrims, to reduce the chances of accidents.\n#Person1#: Pilgrimages are common in many faiths.\n#Person2#: Yes. In England, C

# Model

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2024.12.12: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [10]:
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Mis

# Inference

In [11]:
print(f"Number of test dialogues: {len(test_dialogues)}")
if len(test_dialogues) == 0:
    print("The test_dialogues list is empty. Check how it is being loaded or initialized.")


Number of test dialogues: 30


In [12]:
summarization_prompt_template = """[INST]
Summarize the dialogue mentioned in the user input. Be specific and concise in your summary.
Ensure that you retain the entities mentioned in the dialogue in your summary.

### User Input:
{dialogue}
[/INST]
"""

In [13]:
predicted_summaries = []

In [14]:
for gold_dialogue in tqdm(test_dialogues):

    try:
        prompt = summarization_prompt_template.format(dialogue=gold_dialogue)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            use_cache=True,
            temperature=0,
            pad_token_id=tokenizer.eos_token_id
        )

        prediction = tokenizer.decode(
            outputs[0][inputs.input_ids.shape[-1]:],
            skip_special_tokens=True,
            cleanup_tokenization_spaces=True
        )

        predicted_summaries.append(prediction)

    except Exception as e:
        print(e) # log error and continue
        continue

  3%|▎         | 1/30 [00:00<00:04,  6.52it/s]

'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'


100%|██████████| 30/30 [00:00<00:00, 124.63it/s]

'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' object has no attribute 'attn_bias'
'NoneType' ob

# Evaluation

In [15]:
if len(predicted_summaries) > 1:  # Check if the list has at least 2 elements
    print(predicted_summaries[1])
else:
    print("The list is too short or empty.")


The list is too short or empty.


In [16]:
if len(predicted_summaries) > 1:
    print(predicted_summaries[1])
else:
    print("The list does not have enough elements.")

The list does not have enough elements.


In [17]:
predicted_summaries[1]

IndexError: list index out of range

In [ ]:
bert_scorer = evaluate.load("bertscore")

In [ ]:
score = bert_scorer.compute(
    predictions=predicted_summaries,
    references=test_summaries,
    lang='en',
    rescale_with_baseline=True
)

In [ ]:
sum(score['f1'])/len(score['f1'])